## Let's create a bot....

![Newsbot](https://static.vecteezy.com/system/resources/previews/000/518/290/non_2x/ai-artificial-intelligence-technology-robot-cartoon-001-vector.jpg)

During today's class, we're going to learn how to create a Slack bot. You'll be able to take the bots which you've been prototyping (and hopefully starting to build in code) and turn them into Slack bots. In our next class (after Spring Break 🌴), we'll learn how to move these bots off of our laptop and into the cloud (where they can run 24x7, even while we sleep).

To illustrate some simple bot concepts and how we'll integrate with the Slack APIs and libraries, we're going to build a simple bot related to the coronavirus. Once we learn the Slack bot basics, we'll have you move your bot ideas to Slack.

**Ok, let's get started....**

For our simple bot, we have a few basic **requirements**:

1. We'd like to be able to ask it for the latest statistics on the virus
2. We'd like to be able to ask it for the most recent statistics from different parts of the world
3. We'd like to be able to get the latest news on the coronavirus


e.g.:
```
@covidbot confirmed
@covidbot recovered
@covidbot news about coronavirus
```

We're going to build this bot on Slack but we'd like to easily "port" it to Alexa as well. 

### Before we get to the corona-bot...

Before we get to the coronavirus bot, explore some simple bot concepts using the News API as a source of data. If you are looking to build a simple news bot, this might be helpful!

<img src="https://external-preview.redd.it/5TUMGrd5nfl7nYnA9RosPBR9l1PUMIMeYplMAtE9oNQ.png?width=960&crop=smart&auto=webp&s=bfd95c6af23083aa439eff0b632a40fcd9475ea0" width="800">

We're going to use a nice service call the [News API](https://newsapi.org/). The News API is a service which allows us to: "*Get breaking news headlines, and search for articles from over 30,000 news sources and blogs with our news API*". It provides simple searching access to a large number of news services.

**Please** sign up for an API key - you'll need it in the code below: https://newsapi.org/register

The nice folks at News API have written a [python library](https://newsapi.org/docs/client-libraries/python) so that we can easily call their API. Let's install that now:

In [ ]:
!pip install newsapi-python

Now, let's look at how we can call the News API to get the various news we'll need to handle our bot interactions. Remember, we want out bot to be able to fetch:

1. Top stories
2. The latest stories from a given site or domain name.
3. Search for latest stories given a term/topic.


To get the top headlines from the News API, we can do the following:

In [ ]:
# import the NewsApiClient
from newsapi import NewsApiClient

# put your News API key here
NEWSAPI_KEY = ''

newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

# this is how we call the news api to get the top headlines in the US (english)
top_headlines = newsapi.get_top_headlines(language='en', country='us')

What sort of python object are we working with?

In [ ]:
type(top_headlines)

Let's inspect the dictionary a bit:

In [ ]:
top_headlines.keys()

It looks like we have three fields: `status`, `totalResults` and `articles`. Let's print them out and see what they look like:

In [ ]:
top_headlines['status']

In [ ]:
top_headlines['totalResults']

Let's look at `articles` - what type is it?

In [ ]:
type(top_headlines['articles'])

How would we print out the first article?

In [ ]:
# Your Turn: how would we print out the first article?




The article dictionary has a lot of great info: title, description, url, image, published date and "source".

In [ ]:
# Your Turn: loop over the articles, printing out the title, source name and published date




### Now, let's see how we get the top stories for a given news site:

News API has an API called "[everything](https://newsapi.org/docs/endpoints/everything)" which allows us to pass in a domain and get all of the news from that domain/site. Here is how we get the latest news from the Washington Post:

In [ ]:
# top headlines from WaPo
top_headlines = newsapi.get_everything(domains='washingtonpost.com', language='en')

for article in top_headlines['articles']:
    print(article['title'])
    print(article['source']['name'])
    print(article['publishedAt'])
    print('---'*10)

### Last, how do we get the latest articles for a given search query:

Here is how we'd search for top stories for a given query. In this case, let's look for the news on the "coronavirus":

In [ ]:
query = 'coronavirus'

top_headlines = newsapi.get_top_headlines(q=query, language='en')

for article in top_headlines['articles']:
    print(article['title'])
    print(article['source']['name'])
    print(article['publishedAt'])
    print('---'*10)

## Now, let's get to work on our Slack bot

Now that we have the news-side of our bot (mostly) worked out, let's start on our Slack integration.

**First**, let's create our news bot on Slack:

Make sure you are logged into slack.com with your account you're using for this class and head to [api.slack.com](https://api.slack.com/apps?new_granular_bot_app=1) to create your own bot. Fill out the form by using a clever bot name (you can't all use the same name!) and selecting `Computational Journalism 2020` from the Workspace dropdown. If you don't see "Computational Journalism 2020" in the Workspace dropdown menu, you may need to log back in under a different email/account.

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_create.png" width="500">

**Next**, we need to give our app permissions so that it can post messages to Slack:

Navigate to `OAuth & Permissions` on the sidebar and scroll down to the `Bot Token Scopes` section. `Scopes` give your app permission to do things (for example, post messages). 

Click on `Add an OAth Scope`.

Add the `chat:write` scope to grant your app the permission to post messages in channels it's a member of.

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_scopes.png" width="500">

**Now**, scroll back up to the top of the `OAuth & Permissions` screen and click on the green `Install App to Workspace` button.

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_install.png" width="500">

You should see a prompt that looks like the following - click `Allow` so that your bot can be installed into our Computational Journalism Slack "workplace."

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_allow.png" width="500">

**Important!!**

Copy the `Bot User OAuth Access Token` - this will be our API token! We'll use it in the code below!

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_token.png" width="600">

**Last...**

As we develop our bot, each of us will create a new channel in Slack that we can use as our own "sandbox." We need a place to test out our bot (without annoying our classmates!) so go ahead and create a channel with your name in it, like `mikes-bot` or `marks-bot`.

After creating your channel, go to that channel and invite your bot to the channel. You can do it by typing `/invite` followed by your bot's name. For example, if I wanted to invite `@newsbot` to my channel, I'd type:

```
/invite @newsbot
```

After doing this, you should see a note saying: `Your bot was added to [your channel] by [you].`

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_invite.png">

**Great!!** 🎉🎉

We're almost there! Now, let's install the Slack python library which makes it easy to post message to slack. To install Slack's python library, run the following:

In [ ]:
!pip install slackclient

### Sending our first message to Slack

To send a message from our bot (notebook) to Slack, you can update a few variables below and fire away. You'll need to:

1. add your API token/key, which you copied above, to the `SLACK_API_TOKEN` variable,
2. update the `channel` variable with the name of the channel that you just created. Make sure that it starts with `#`. For example, `#mike-test`.
3. feel free to update the `slack_message` variable - this is what your bot will post to your channel.



In [ ]:
import slack

# our slack api token
# copy this from the OAuth & Permissions --> "Bot User OAuth Access Token"
SLACK_API_TOKEN = ""

# initialize our slack api client
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

# put your slack channel here, like #mike-test
channel = ""
slack_message = "Hello!"

# post a message to a channel
response = slack_client.chat_postMessage(channel=channel, text=slack_message)

print(response)

### Do you get an error?

If you see an error that says something like `The request to the Slack API failed`, you may not have completed your bot setup properly. 

```
SlackApiError: The request to the Slack API failed.
The server responded with: {'ok': False, 'error': 'missing_scope', 'needed': 'chat:write:bot', 'provided': 'calls:write'}
```


Most likely, you'll see an error message that says the following:

```
RuntimeError: This event loop is already running
```

To make this go away, we're going to add a bit of code to get around this error. Let's install this library first:

In [ ]:
# https://github.com/erdewit/nest_asyncio
!pip install nest_asyncio

In [ ]:
import slack

# to play nicely with Jupyter notebook
import asyncio
import nest_asyncio

nest_asyncio.apply()
# end of playing nice

# initialize our slack api client
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

# put your slack channel here, like #mike-test
channel = ""
slack_message = "Hello!"

# post a message to a channel
response = slack_client.chat_postMessage(channel=channel, text=slack_message)

print(response)

### Great! Now, let's make something a bit more useful...

Let's imagine we want to have our newsbot send us top headlines a few times a day...alertings us when the News API has a new top headline. 

To do this, how might we have our bot run forever, waking up every so often? The following code will loop "forever", pausing for 5 seconds each time it runs:

In [ ]:
import time

# a loop that runs "forever"
while True:

    print("hello")

    # sleep for 5 seconds
    time.sleep(5)

To stop this, make sure you click the Stop icon `[]` in the notebook menu.

**Detour on while loops**

We have already seen `for` loops that iterate over a fixed set of items, say elements in a list. A `while` loop continues executing until the condition its testing for is no longer true. To make this real, let's consider tossing coins. We can simulate a coin toss with the `choice()` function in the `random` package.

In [ ]:
from random import choice

# a coin is just a list with two options - heads or tails
coin = ["Heads","Tails"]

# a toss is a random selection between these choices
toss = choice(coin)

# have a look -- do this several times
toss

Now, we can use the `while` loop toss a coin until we see five heads in total. We'll stop once we see the fifth head. Here's a simple loop.

In [ ]:
total_tosses = 0
total_heads = 0

while total_heads < 5:
    
    # keep track of the number of times we toss the coin
    total_tosses += 1
    
    # toss it
    toss = choice(coin)
    
    # test if its heads, and if so, increment the number of heads we've seen
    
    if toss == "Heads":
        total_heads += 1
        
# print out how many tosses we needed to see 5 heads

print(total_tosses)

**Back to our bot**

Ok, let's expand on that a bit and post a message to our Slack channel every few seconds. We're introducing a new python library here called the [datetime](https://docs.python.org/3/library/datetime.html) library. This is a very powerful library for dealing with, well, dates and times. We simply using it to print out the current time ("now").

In [ ]:
from datetime import datetime

# we're assuming you set the channel variable in a cell above
#channel = "#mike-test"

while True:

    # create a message that says "hello, the time is now 2020-03-11 12:34:00"
    slack_message = "hello, the time is now " + str(datetime.now())

    # post the message to our Slack channel
    response = slack_client.chat_postMessage(channel=channel, text=slack_message)

    print(response)
    
    # sleep for 5 seconds
    time.sleep(5)

Again, make sure you hit the Stop button above to have your bot stop spamming you!


Let's update our simple bot just a bit and post the top headline from the News API each time it runs:

In [ ]:
# let's call the news api for the top headlines
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

# let's loop forever
while True:

    # get the top headline from the news api
    top_headlines = newsapi.get_top_headlines(language='en', country='us')

    # get the first headline in the list
    top_headline = top_headlines['articles'][0]

    # build a slack message with the title and url
    slack_message = top_headline['title'] + ' ' + top_headline['url']

    # post the headline to our slack channel
    response = slack_client.chat_postMessage(channel=channel, text=slack_message)

    # sleep for 5 seconds
    time.sleep(5)
    

Well, that's not quite right! 🙃 We don't want out bot spamming us with the same headline every 5 seconds! What we'd really like is for our bot to post a headline to Slack only when it changes (i.e. when there is a new top headline). What's a technique that we can use to make sure we don't send the message over and over again?

In [ ]:
# let's modify the code to only post a message to slack when we have a new message

newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

# keep track of the last/previous message we sent to slack
# we will initialize this to an empty string
previous_message = ""

while True:

    # get the top headline from the news api
    top_headlines = newsapi.get_top_headlines(language='en', country='us')

    top_headline = top_headlines['articles'][0]

    # build a slack message with the title and url
    slack_message = top_headline['title'] + ' ' + top_headline['url']

    # have we sent this one before?
    if slack_message == previous_message:
        print('we have already sent this message: ' + slack_message)
    else:
        # it's a new article! let's post it to our slack channel
        response = slack_client.chat_postMessage(channel=channel, text=slack_message)

        # save the message we just sent as our "previous" (or last) message
        previous_message = slack_message
    
    # sleep for 5 seconds
    time.sleep(5)
    

**Notice** that we still have a problem with sending the same messages if we stop and start out bot. How might we fix that? Where can we keep track of the previous messages that we've sent? Something that would be persistant in-between the bot stopping and starting?

## Making our Bot Interactive

So far, we have a simple newsbot that posts top news headlines to our Slack channel. This is a nice start but we'd really like to query our bot on demand to ask for top headlines, news from a particual site or even news about a particular topic. I won't go as far as calling it "conversational" but I'd like our bot to be more interactive. To be able to talk back-and-forth with our Slack bot, we have two options. We can use:
* [Slack Events](https://api.slack.com/events-api), or
* [Real Time Messaging](https://slack.dev/python-slackclient/real_time_messaging.html)

Today, we're going to use Slack Events, which allows us to register for certain events we'd like to hear about (a Slack user sends our bot a message, or a someone adds an emoji to a Slack message, for example). When one of the events occurs, Slack will send us a message, letting us know about it. How do they send us the messages you might ask? Well, they send them to us as HTTP requests. Remember from a few lectures back we talked about making HTTP reqeusts? A client (a browser, some python code we write, etc) makes a request for some page or API on a server and the server sends back a response. Is this ringing a bell? Well, in this case, Slack will be sending us HTTP requests each time there an Event occurs that we've asked to be notified about. This means that we get to write our own web service! To do this, we'll be using a python library called [Flask](https://palletsprojects.com/p/flask/). 

[Flask](https://palletsprojects.com/p/flask/) is a "*lightweight web application framework*" that makes it easy to write web applications/APIs/web services. Let's write a simple one now and we'll get back to Slack in a minute.

To get started, install the flask library:

In [ ]:
!pip install flask

The code below creates a simple web service which will "serve" request to the following URLs:

* http://127.0.0.1:5000/
* http://127.0.0.1:500/api

You may notice something new in the code below - the `@app.route("/")` above some of the functions. This is a called a "decorator" and it essentially extends the behavior of our functions. In this case, the `@app.route` decorator is code that exists in the Flask library and it handles a lot of the hard work of making sure what whenever a request is made to, say, `/api`, that we run the code found in the `def our_api()` method. You can read more about python decorators here: https://realpython.com/primer-on-python-decorators/

In [ ]:
from flask import Flask

# This `app` represents our Flask app
app = Flask(__name__)

# we can create "routes" which tell our app where to "route" incoming requests
@app.route("/")
def hello():
    return "Hello there!"

# this is a simple "/api" endpoint
@app.route("/api")
def our_api():
    return "If we had an API, it could go here!"

# start our Flask app and have it run on port 5000
app.run(port=5000)

**When you run the cell above, you should see:**

```
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```


What is `127.0.0.1`? And, what's the `5000`?

`127.0.0.1`, also called "[localhost](https://en.wikipedia.org/wiki/Localhost)", is the IP address of our computer.


![There's no place like 127.0.0.1](https://bookofjoe.typepad.com/photos/uncategorized/2008/12/07/1htrd.jpg "There's no place like 127.0.0.1")

As for the `5000` - that is the port number assigned to our Flask application. We communicate with our app by connecting to it via the IP address (127.0.0.1) and port number.

Well-known ports include 80 for ordinary HTTP and 443 for HTTPS traffic. A port can be any number from 0 to 65535. The low numbers (below 1024) are reserved for your computer and are managed by an administrator. Ports from [1024 through 49151 are "registered" by IANA](https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.xhtml), the Internet Assigned Numbers Authority. [Here's a nice list of well-known and registered ports](https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers). (Look for 8888 and you'll see it's associated with our notebooks! Oh and look at this browser window's navigation bar to see 8888 in the address of this notebook.) Now, numbers beyond these ranges are a zoo... they are for custom, often temporary communication. They are referred to as "private/dynamic". 


### Creating a connection from Slack to our Bot

Now, we'll be running the bot code on our laptop. But Slack requires bots to run behind a public HTTPS server. To accomplish this easily, we'll need some help from a tool called ngrok which will route information to and from Slack for us.

At a technical level, ngrok is a command-line program that opens a secure tunnel to localhost and exposes that tunnel behind an HTTPS endpoint. ngrok makes it so Slack can talk to your code right away. 

<img src="https://cloud.githubusercontent.com/assets/32463/25376866/940435fa-299d-11e7-9ee3-08d9427417f6.png">

Follow the next three steps to install and run ngrok:

1. [Download](https://ngrok.com/download) the ngrok client for your operating system.
2. Unzip it to a location you can remember.
3. Open up a new terminal, cd into the location, and enter:


on Unix/Mac:

      ./ngrok http 5000

on Windows:

      ngrok.exe http 5000
What you should see is something like this.

<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/ng.jpg" style="width: 50%; border: #000000 1px outset;"/>

NOTE The important part of the information here is the Forwarding https address - in this case: `https://3c825e6b.ngrok.io`. You will need this in the next step.

### Slack Events

First, we need to enable Slack Events in our app settings and have Slack "verify" us, before we're able to receive events from Slack. To do this, we need to build upon our simple Flask app and create an api endpoint (our "route") that Slack can call to verify that we are who we say we are. We'll do this using a python library developed by the folks at Slack. Let's install that now:

In [ ]:
!pip install slackeventsapi

We need to get our Slack signing secret, which will be used to verify our bot with Slack. This is similar to an API key but slightly different. To find it, you can:

1. Go back to our [Slack App settings](https://api.slack.com/apps), and
2. Navigate to the `Basic Information` section
3. Scroll down to `Signing Secret` under the `App Credentials` section and hit the "Show" button. 
4. Copy the Signing Secret and paste it below in the `SLACK_SIGNING_SECRET` variable. 


<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_secret.png" width="600">

Once you've done that, run the cell below to start up our Slack bot so that Slack can verify that we are legit!

In [ ]:
import pprint
from flask import Flask
from slackeventsapi import SlackEventAdapter

# This `app` represents our Flask app
app = Flask(__name__)

# slack signing secret found in: https://api.slack.com/apps
# which is the "Basic Information" tab --> App Credential section --> Show Signing Secret
SLACK_SIGNING_SECRET = ""

slack_events_adapter = SlackEventAdapter(SLACK_SIGNING_SECRET, "/slack/events", app)

# Start the app on port 5000
app.run(port=5000)

Once the above is running, we have to do a few more steps to tell Slack where to reach us. Please do the following to make sure Slack can "verify" our bot:

1. Head back to https://api.slack.com/apps/ and navigate to `Event Subscriptions`. Toggle it on.
2. In the Request URL field, enter your ngrok http URL (from above) and append `/slack/events` to the end of it. It should look like: https://e90229c5.ngrok.io/slack/events
3. Click "Save Changes"

If you see `Request URL Your URL didn't respond with the value of the challenge parameter`, make sure our bot is running below and then hit the Retry button.

We're looking for a Verified ✅ next to the Request URL. Once we have this, we can move on!

A quick note about the code above...it looks similar to our Flask app from above, but we're leveraging some additional functionality provided by the Slack Events API python library. By running the code above, our Flask app exposed an api/endpoint `http://127.0.0.1:5000/slack/events` which handles the Slack verification process for us. We'll see in a minute how this library will allow us to easily route Slack Events (e.g. someone sent my by an @mention) to code in our Flask app.

### Enabling @bot Mention Events

Now that our service is verified, we can start to add Events that we'd like to receive from Slack. Since we're building a simple bot that we'd like users to interact with, let's create an Event that allows us to get a message each time someone mentions our `@newsbot` in our Slack channel. To do this:

1. Head back to our Slack app settings page and click on `Event Subscriptions` in the left-hand menu
2. Click on `Subscribe to bot events`
3. Click the `Add Bot User Event` button
4. Search for `app_mention` and add that event
5. Click the green `Save Changes` button in the bottom right-hand side of the page
6. Click on the `reinstall your app` link at the top of the screen (in the yellow message). This will redirect you back to your App Installation page. Click `Allow` which will allow your bot to have permissions for the newly create Event(s).

**Now** let's modify our bot code to receive messages whenever our bot is @mentioned by a user. We can do this by creating a function that uses the decorator:
```
@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    print "we got an app mention!"
```

The code below will simply print out the data we receive from Slack when our bot gets an @mention. Run the following code and then send your bot some messages in your Slack channel!

In [ ]:
import pprint

@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    
    # lets pretty-print the data we receive from slack
    pprint.pprint(event_data)

# Start the app on port 5000
app.run(port=5000)

Now that we're able to receive @mentions from Slack, let's update our code to send a message back to the channel where the @mention came from. To do this, we'll use the slack "WebClient" that we used earlier in class. In this case, we'll simply extract the channel ID from the Slack message and send a quick message back:

In [ ]:
# let's add back our slack WebClient
# which allows us to post messages back to Slack
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    
    # pretty-print the data we receive from slack
    pprint.pprint(event_data)

    # lets get the channel ID from the incoming message
    # and send "Hi there!" back to that channel
    channel = event_data["event"]["channel"]
    
    response = slack_client.chat_postMessage(channel=channel, text="Hi there!")

# Start the app on port 5000
app.run(port=5000)

### A COVID19 bot

We are going to pull data from a collection being [collected by Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19). There are several such collections, but this has a lot to recommend it. It is updated daily in a series of CSVs that have the current date in their title. Let's pull today's.

In [ ]:
from pandas import read_csv

date = "03-10-2020"

covid = read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+date+".csv")
covid.head()

You can see that this contains a fair bit of information. You can answer questions about counts of confirmed cases, deaths and recoveries, broken down by countries, states and some other areas. We can create `.sum()`'s to count cases worldwide. 

In [ ]:
confirmed = covid["Confirmed"].sum()
confirmed

In [ ]:
deaths = covid["Deaths"].sum()
deaths

In [ ]:
recovered = covid["Recovered"].sum()
recovered

We can now create a simple bot that when you mention its name, along with keywords like "confirmed" or "deaths" or "recoveries" and get back the current counts.

**Important:** This code should be the basis of your Slack bot - so please extend/modify this code. As a reminder, you will need to do a few quick setups to get this working in between closing your notebook, restarting your laptop, etc. 


**Step 1**: Make sure ngrok is running. The instructions are above but as a reminder: open up a new terminal, cd into the location where you downloaded it, and enter:

on Unix/Mac:

      ./ngrok http 5000

on Windows:

      ngrok.exe http 5000

What you should see is something like this.

<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/ng.jpg" style="width: 50%; border: #000000 1px outset;"/>

NOTE The important part of the information here is the Forwarding https address - in this case: `https://3c825e6b.ngrok.io`. You will need this in the next step.


**Step 2**: Go back to your Slack app settings to update the ngrok.io address (if has changed since hte last time you configured your Slack bot settings). Head to https://api.slack.com/apps, click on your bot's name, then click on `Event Subscriptions` on the left-hand menu. On the Event Subscriptions page, enter your ngrok.io url in the `Request URL` field. Remember to add `/slack/events` to the end of your URL so it should look like: `https://3c825e6b.ngrok.io/slack/events`

**Step 3**: Find your Slack API Token and Signing Secret that you'll use in the code below. 

The Slack API Token can be found on your slack app settings page and then clicking on `OAuth & Permissions` --> `Bot User OAuth Access Token`.

The Slack Signing Token can be found on your slack app settings page and then clicking on `Basic Information`, scoll down to the `App Credentials` section --> `Signing Secret` (you need to click Show and then cut/paste).

In [ ]:
import pprint
import datetime
import slack
from pandas import read_csv
from flask import Flask
from slackeventsapi import SlackEventAdapter

# our slack api token
# copy this from the OAuth & Permissions --> "Bot User OAuth Access Token"
SLACK_API_TOKEN = ""

# slack signing secret found in: https://api.slack.com/apps
# which is the "Basic Information" tab --> App Credential section --> Show Signing Secret
SLACK_SIGNING_SECRET = ""

# This `app` represents our Flask app
app = Flask(__name__)

# let's add back our slack WebClient
# which allows us to post messages back to Slack
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

slack_events_adapter = SlackEventAdapter(SLACK_SIGNING_SECRET, "/slack/events", app)

@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    
    # pull out the text of the message to the bot
    text = event_data["event"]["text"]
    
    # and pull the channel we need to post a response to
    channel = event_data["event"]["channel"]
   
    # this is a cheap way to get data from yesterday
    dt = str(datetime.datetime.today()-datetime.timedelta(days=1) )
    date = dt[5:10]+"-"+dt[:4]

    # pull the data from JHU
    covid = read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+date+".csv")
   
    # and compute some simple things...
    confirmed = covid["Confirmed"].sum()
    deaths = covid["Deaths"].sum()
    recovered = covid["Recovered"].sum()

    
    if "confirmed" in text.lower():       
        response = slack_client.chat_postMessage(channel=channel, text="Confirmed: "+str(confirmed))

    if "death" in text.lower():
        response = slack_client.chat_postMessage(channel=channel, text="Deaths: "+str(deaths))

    if "recover" in text.lower():
        response = slack_client.chat_postMessage(channel=channel, text="Recovered: "+str(recovered))

# Start the app on port 5000
app.run(port=5000)

From here, the world is your oyster! You can respond in a variety of ways and scan the input from a user to the bot in different ways! Try it!